# ANOVA (Análisis de Varianza)

## Objetivos
- Entender cuándo usar ANOVA vs múltiples pruebas t
- Realizar ANOVA de un factor
- Verificar supuestos (normalidad, homogeneidad de varianzas)
- Aplicar pruebas post-hoc (Tukey, Bonferroni)
- Calcular tamaño del efecto (η²)
- Usar alternativas no paramétricas (Kruskal-Wallis)

---

## 1. Preparación

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

# Cargar datos
df = pd.read_csv('../datos/ejemplo_satisfaccion_clientes.csv')
print(f"Dataset cargado: {df.shape[0]} registros")
print(f"Áreas disponibles: {df['area'].unique()}")

## 2. Recordatorio: ¿Por qué ANOVA?

### Problema con múltiples pruebas t:

Si queremos comparar 4 áreas, necesitaríamos **6 pruebas t**:
- Norte vs Sur
- Norte vs Este  
- Norte vs Oeste
- Sur vs Este
- Sur vs Oeste
- Este vs Oeste

❌ **Problema:** Con α=0.05, la probabilidad de Error Tipo I se infla.

✅ **Solución:** ANOVA hace **UNA sola prueba** que controla el error global.

### Pregunta de Investigación:
> ¿La satisfacción promedio es diferente entre las 4 áreas de servicio?

---

## 3. Análisis Descriptivo por Grupos

In [ ]:
# Estadísticos descriptivos por área
resumen = df.groupby('area')['satisfaccion'].agg([
    ('n', 'count'),
    ('Media', 'mean'),
    ('Mediana', 'median'),
    ('Desv.Std', 'std'),
    ('Mínimo', 'min'),
    ('Q1', lambda x: x.quantile(0.25)),
    ('Q3', lambda x: x.quantile(0.75)),
    ('Máximo', 'max')
]).round(2)

print("="*80)
print("ESTADÍSTICOS DESCRIPTIVOS POR ÁREA")
print("="*80)
print(resumen)

# Media general
media_general = df['satisfaccion'].mean()
print(f"\nMedia General: {media_general:.2f}")

## 4. Visualización Inicial

In [ ]:
# Boxplot comparativo
plt.figure(figsize=(12, 6))
df.boxplot(column='satisfaccion', by='area', patch_artist=True, grid=False)
plt.suptitle('')  # Eliminar título automático
plt.title('Distribución de Satisfacción por Área', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Satisfacción (1-10)', fontsize=12)
plt.axhline(y=media_general, color='red', linestyle='--', linewidth=2, label='Media General')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Violin plot
plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x='area', y='satisfaccion', palette='Set2')
plt.title('Distribución de Satisfacción por Área (Violin Plot)', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Satisfacción (1-10)', fontsize=12)
plt.axhline(y=media_general, color='red', linestyle='--', linewidth=1.5, label='Media General')
plt.legend()
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de medias con intervalos de confianza
medias = df.groupby('area')['satisfaccion'].mean()
std_errors = df.groupby('area')['satisfaccion'].sem()

plt.figure(figsize=(10, 6))
medias.plot(kind='bar', yerr=std_errors*1.96, capsize=5, color='steelblue', edgecolor='black')
plt.title('Media de Satisfacción por Área (IC 95%)', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Satisfacción Promedio', fontsize=12)
plt.xticks(rotation=0)
plt.axhline(y=media_general, color='red', linestyle='--', linewidth=2, label='Media General')
plt.ylim(0, 10)
plt.legend()
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 5. Verificación de Supuestos

### Supuesto 1: Normalidad

Cada grupo debe seguir una distribución normal.

In [ ]:
print("="*60)
print("PRUEBA DE NORMALIDAD (SHAPIRO-WILK)")
print("="*60)

normalidad_ok = True
for area in df['area'].unique():
    datos = df[df['area'] == area]['satisfaccion']
    stat, p = stats.shapiro(datos)
    resultado = "✓ Normal" if p > 0.05 else "⚠️ No normal"
    print(f"{area:10s}: W={stat:.4f}, p={p:.4f} {resultado}")
    
    if p <= 0.05:
        normalidad_ok = False

if normalidad_ok:
    print("\n✓ Todos los grupos cumplen supuesto de normalidad")
else:
    print("\n⚠️ Algunos grupos NO cumplen normalidad")
    print("   Considerar:")
    print("   1. Transformación de datos (log, raíz cuadrada)")
    print("   2. Prueba no paramétrica (Kruskal-Wallis)")

In [ ]:
# Q-Q plots para cada grupo
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
areas = df['area'].unique()
axes = axes.flatten()

for i, area in enumerate(areas):
    datos = df[df['area'] == area]['satisfaccion']
    stats.probplot(datos, dist="norm", plot=axes[i])
    axes[i].set_title(f'Q-Q Plot: {area}', fontweight='bold')
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Supuesto 2: Homogeneidad de Varianzas

Las varianzas de los grupos deben ser similares.

In [ ]:
# Prueba de Levene
grupos = [df[df['area'] == area]['satisfaccion'] for area in df['area'].unique()]
stat_levene, p_levene = stats.levene(*grupos)

print("="*60)
print("PRUEBA DE HOMOGENEIDAD DE VARIANZAS (LEVENE)")
print("="*60)
print(f"Estadístico: W={stat_levene:.4f}")
print(f"p-value: {p_levene:.4f}")

if p_levene > 0.05:
    print("\n✓ Varianzas HOMOGÉNEAS (usar ANOVA estándar)")
    varianzas_ok = True
else:
    print("\n⚠️ Varianzas NO homogéneas")
    print("   Considerar Welch's ANOVA (no asume varianzas iguales)")
    varianzas_ok = False

In [ ]:
# Visualizar varianzas
varianzas = df.groupby('area')['satisfaccion'].var()

plt.figure(figsize=(10, 6))
varianzas.plot(kind='bar', color='coral', edgecolor='black')
plt.title('Varianzas por Área', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Varianza', fontsize=12)
plt.xticks(rotation=0)
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 6. ANOVA de Un Factor

### Hipótesis:
- **H₀:** μ₁ = μ₂ = μ₃ = μ₄ (Todas las medias son iguales)
- **H₁:** Al menos una media es diferente

In [ ]:
# Opción 1: ANOVA con scipy
norte = df[df['area'] == 'Norte']['satisfaccion']
sur = df[df['area'] == 'Sur']['satisfaccion']
este = df[df['area'] == 'Este']['satisfaccion']
oeste = df[df['area'] == 'Oeste']['satisfaccion']

f_stat, p_value = stats.f_oneway(norte, sur, este, oeste)

print("="*60)
print("ANOVA DE UN FACTOR")
print("="*60)
print(f"Estadístico F: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print("="*60)

In [ ]:
# Opción 2: ANOVA con statsmodels (tabla completa)
modelo = ols('satisfaccion ~ C(area)', data=df).fit()
tabla_anova = sm.stats.anova_lm(modelo, typ=2)

print("\nTabla ANOVA Completa:")
print(tabla_anova)

print("\n📊 Explicación de la tabla:")
print("   sum_sq: Suma de cuadrados")
print("   df: Grados de libertad")
print("   F: Estadístico F")
print("   PR(>F): p-value")

In [ ]:
# Decisión
alpha = 0.05

print("\n" + "="*60)
print("DECISIÓN")
print("="*60)

if p_value < alpha:
    print(f"p-value ({p_value:.4f}) < α ({alpha})")
    print("✗ RECHAZAMOS H₀")
    print("\n💡 CONCLUSIÓN:")
    print("   Al menos una área tiene satisfacción promedio DIFERENTE")
    print("   Siguiente paso: Pruebas POST-HOC para identificar cuál(es)")
    realizar_posthoc = True
else:
    print(f"p-value ({p_value:.4f}) ≥ α ({alpha})")
    print("✓ NO RECHAZAMOS H₀")
    print("\n💡 CONCLUSIÓN:")
    print("   No hay evidencia suficiente de diferencias entre áreas")
    print("   Las medias son estadísticamente similares")
    realizar_posthoc = False

## 7. Tamaño del Efecto (Eta Cuadrado - η²)

In [ ]:
# Calcular Eta cuadrado
ss_between = tabla_anova['sum_sq']['C(area)']
ss_total = tabla_anova['sum_sq'].sum()
eta_squared = ss_between / ss_total

print("="*60)
print("TAMAÑO DEL EFECTO")
print("="*60)
print(f"Eta² (η²): {eta_squared:.4f}")
print(f"\nInterpretación: {eta_squared*100:.1f}% de la varianza en satisfacción")
print(f"                es explicada por el área")

# Interpretación del tamaño
if eta_squared < 0.01:
    print("\nEfecto: PEQUEÑO")
elif eta_squared < 0.06:
    print("\nEfecto: MEDIANO")
else:
    print("\nEfecto: GRANDE")

print("\nEscala:")
print("   η² < 0.01: Pequeño")
print("   0.01 ≤ η² < 0.06: Mediano")
print("   η² ≥ 0.06: Grande")

## 8. Pruebas Post-Hoc (Si rechazamos H₀)

Si ANOVA es significativo, usamos **pruebas post-hoc** para determinar **qué pares** de grupos son diferentes.

### Prueba de Tukey (HSD)

In [ ]:
if realizar_posthoc:
    print("="*80)
    print("PRUEBA POST-HOC: TUKEY HSD")
    print("="*80)
    
    # Realizar prueba de Tukey
    tukey = pairwise_tukeyhsd(endog=df['satisfaccion'], groups=df['area'], alpha=0.05)
    
    print(tukey)
    
    # Visualización
    tukey.plot_simultaneous(figsize=(10, 6))
    plt.title('Intervalos de Confianza 95% - Prueba de Tukey', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("No se requieren pruebas post-hoc (ANOVA no significativo)")

In [ ]:
if realizar_posthoc:
    # Resumen de diferencias significativas
    print("\n📊 RESUMEN DE DIFERENCIAS SIGNIFICATIVAS:")
    print("="*80)
    
    tukey_table = tukey.summary()
    for i, row in enumerate(tukey_table.data[1:]):
        grupo1, grupo2, meandiff, p_adj, lower, upper, reject = row
        
        if reject:  # Si hay diferencia significativa
            direccion = "mayor" if meandiff > 0 else "menor"
            print(f"\n• {grupo1} vs {grupo2}:")
            print(f"  Diferencia de medias: {abs(meandiff):.2f}")
            print(f"  p-value ajustado: {p_adj:.4f}")
            print(f"  {grupo1} tiene satisfacción {direccion} que {grupo2}")

### Prueba de Bonferroni (alternativa más conservadora)

In [ ]:
if realizar_posthoc:
    print("\n" + "="*80)
    print("PRUEBA POST-HOC: BONFERRONI")
    print("="*80)
    
    mc = MultiComparison(df['satisfaccion'], df['area'])
    resultado_bonf = mc.allpairtest(stats.ttest_ind, method='bonf')
    print(resultado_bonf[0])

## 9. Alternativa No Paramétrica: Kruskal-Wallis

Si **NO se cumplen** los supuestos de normalidad, usamos **Kruskal-Wallis** (ANOVA no paramétrico).

In [ ]:
# Prueba de Kruskal-Wallis
h_stat, p_kw = stats.kruskal(norte, sur, este, oeste)

print("="*60)
print("PRUEBA DE KRUSKAL-WALLIS (NO PARAMÉTRICA)")
print("="*60)
print(f"Estadístico H: {h_stat:.4f}")
print(f"p-value: {p_kw:.4f}")

if p_kw < 0.05:
    print("\n✗ Rechazamos H₀")
    print("   Al menos una área tiene MEDIANA diferente")
    print("\n   Post-hoc: Usar Mann-Whitney U con corrección Bonferroni")
else:
    print("\n✓ No rechazamos H₀")
    print("   No hay diferencia entre medianas")

In [ ]:
# Post-hoc para Kruskal-Wallis (Mann-Whitney U con Bonferroni)
if p_kw < 0.05:
    from itertools import combinations
    
    areas = df['area'].unique()
    n_comparaciones = len(list(combinations(areas, 2)))
    alpha_bonf = 0.05 / n_comparaciones  # Corrección de Bonferroni
    
    print(f"\n📊 POST-HOC: MANN-WHITNEY U (α ajustado = {alpha_bonf:.4f})")
    print("="*80)
    
    for area1, area2 in combinations(areas, 2):
        grupo1 = df[df['area'] == area1]['satisfaccion']
        grupo2 = df[df['area'] == area2]['satisfaccion']
        
        u_stat, p_mw = stats.mannwhitneyu(grupo1, grupo2, alternative='two-sided')
        
        significativo = "✗" if p_mw < alpha_bonf else "✓"
        print(f"{area1} vs {area2}: U={u_stat:.1f}, p={p_mw:.4f} {significativo}")

## 10. Comparación: ANOVA vs Kruskal-Wallis

In [ ]:
print("="*60)
print("COMPARACIÓN DE PRUEBAS")
print("="*60)
print(f"ANOVA (paramétrica):          F={f_stat:.4f}, p={p_value:.4f}")
print(f"Kruskal-Wallis (no param.):   H={h_stat:.4f}, p={p_kw:.4f}")

print("\n💡 ¿Cuál usar?")
if normalidad_ok and varianzas_ok:
    print("   ✓ Usar ANOVA (supuestos cumplidos)")
else:
    print("   ⚠️ Preferir Kruskal-Wallis (supuestos no cumplidos)")

## 11. Análisis Adicional: Otra Variable

In [ ]:
# Comparar calidad_atencion entre áreas
print("="*60)
print("ANÁLISIS ADICIONAL: CALIDAD DE ATENCIÓN POR ÁREA")
print("="*60)

# Descriptivos
resumen_calidad = df.groupby('area')['calidad_atencion'].agg(['count', 'mean', 'std']).round(2)
print("\nEstadísticos Descriptivos:")
print(resumen_calidad)

# ANOVA
grupos_calidad = [df[df['area'] == area]['calidad_atencion'] for area in df['area'].unique()]
f_cal, p_cal = stats.f_oneway(*grupos_calidad)

print(f"\nANOVA: F={f_cal:.4f}, p={p_cal:.4f}")

if p_cal < 0.05:
    print("✗ Hay diferencias significativas en calidad de atención entre áreas")
else:
    print("✓ No hay diferencias significativas en calidad de atención entre áreas")

In [ ]:
# Visualización
plt.figure(figsize=(12, 6))
df.boxplot(column='calidad_atencion', by='area', patch_artist=True, grid=False)
plt.suptitle('')
plt.title('Calidad de Atención por Área', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Calidad de Atención (1-10)', fontsize=12)
plt.tight_layout()
plt.show()

## 12. Reporte Ejecutivo

In [ ]:
print("="*70)
print("REPORTE EJECUTIVO - ANOVA")
print("="*70)

print("\n🎯 OBJETIVO:")
print("   Comparar la satisfacción promedio entre las 4 áreas de servicio")

print("\n📊 DESCRIPTIVOS:")
for area in resumen.index:
    print(f"   • {area}: Media={resumen.loc[area, 'Media']:.2f}, "
          f"SD={resumen.loc[area, 'Desv.Std']:.2f}, n={int(resumen.loc[area, 'n'])}")

print("\n🔬 RESULTADOS ANOVA:")
print(f"   F({int(tabla_anova.loc['C(area)', 'df'])}, {int(tabla_anova.loc['Residual', 'df'])}) = {f_stat:.4f}")
print(f"   p-value = {p_value:.4f}")
print(f"   η² = {eta_squared:.4f} ({eta_squared*100:.1f}% de varianza explicada)")

print("\n💡 CONCLUSIÓN:")
if p_value < 0.05:
    print("   Las áreas tienen satisfacción promedio DIFERENTE")
    
    if realizar_posthoc:
        # Identificar mejor y peor área
        mejor_area = medias.idxmax()
        peor_area = medias.idxmin()
        
        print(f"\n   🏆 Mejor desempeño: {mejor_area} (Media={medias.max():.2f})")
        print(f"   ⚠️ Peor desempeño: {peor_area} (Media={medias.min():.2f})")
        print(f"\n   📌 Recomendación: Replicar buenas prácticas del {mejor_area} en otras áreas")
else:
    print("   No hay diferencia significativa entre áreas")
    print("   La satisfacción es consistente en todas las ubicaciones")

print("\n" + "="*70)

## 13. Ejercicios Propuestos

### Ejercicio 1: Tiempo de Espera
Compara `tiempo_espera` entre las 4 áreas. ¿Hay diferencias significativas?

### Ejercicio 2: Grupos de Edad
Crea 3 grupos de edad:
- Joven: < 35 años
- Adulto: 35-55 años
- Mayor: > 55 años

Compara satisfacción entre estos grupos.

### Ejercicio 3: Verificar Supuestos
Para las comparaciones anteriores, verifica supuestos de normalidad y homogeneidad.

### Ejercicio 4: Análisis Completo
Realiza un análisis completo (ANOVA + post-hoc + visualizaciones) para `calidad_atencion` por `area`.


In [ ]:
# Tu código aquí


---

## Resumen

En este notebook aprendiste a:
- ✓ Realizar ANOVA de un factor
- ✓ Verificar supuestos (normalidad con Shapiro-Wilk, homogeneidad con Levene)
- ✓ Interpretar el estadístico F y p-value
- ✓ Calcular tamaño del efecto (η²)
- ✓ Aplicar pruebas post-hoc (Tukey, Bonferroni)
- ✓ Usar alternativa no paramétrica (Kruskal-Wallis)
- ✓ Crear visualizaciones efectivas (boxplots, violin plots)
- ✓ Interpretar resultados en contexto de negocio

**Siguiente notebook:** Regresión Lineal y Correlación
